# Catalyst Cooperative Jupyter Notebook Template
This notebook lays out a standard format and some best practices for creating interactive / exploratory notebooks which can be relatively easily shared between different PUDL users, and turned into reusable Python modules for integration into our underlying Python packages.

## Begin with a narrative outline
Each notebook should start with a brief explanation (in Markdown) explaining the purpose of the analysis, and outlining the different stages / steps which are taken to accomplish the analysis.
As the analysis develops, you can add new sections or details to this section.

## Notebooks should be runnable
Insofar as it's possible, another PUDL user who has cloned the repository that the notebook is part of should be able to update their `pudl-dev` conda environment, open the notebook, and run all cells successfully.
If there are required data or other prerequisites that the notebook cannot manage on its own -- like a file that needs to be downloaded by hand and placed in a particular location -- those steps should be laid out clearly at the beginning of the notebook.

## Avoid Troublesome Elements

### Don't hardcode passwords or access tokens
Most of our work is done in public Github repositories.
No authentication information should ever appear in a notebook.
These values can be stored in environment variables on your local computer.

### Do not hardocde values, especially late in the notebook
If the analysis depends on particular choices of input values, those should be called out explicitly at the beginning of the notebook.
(NB: We should explore ways to parameterize notebooks, [papermill](https://papermill.readthedocs.io/en/latest/) is one tool that does this).

### Don't hardcode absolute file paths
If anyone is going to be able to use the notebook, the files it uses will need to be stored somewhere that makes sense on both your and other computers.
We assume that anyone using this template has the PUDL package installed, and has a local PUDL data management environment set up.
  * Input data that needs to be stored on disk and accessed via a shared location should be saved under `<PUDL_IN>/data/local/<data_source>/`.
  * Intermediate saved data products (e.g. a pickled result of a computationally intensive process) and results should be saved to a location relative to the notebook, and within the directory hierarchy of the repository that the notebook is part of.
  
### Don't require avoidable long-running computations
Consider persisting to disk the results of computations that take more than a few minutes, if the outputs are small enough to be checked into the repository and shared with other users.
Only require the expensive computation to be run if this pre-computed output is not available.

### Don't litter
Don't leave lots of additional code laying around, even commented out, "just in case" you want to look at it again later.
Notebooks need to be relatively linear in the end, even though the thought processes and exploratory analyses that generate them may not be.
Once you have a working analysis, either prune those branches, or encapsulate them as options within functions.

### Don't load unneccesary libraries
Only import libraries which are required by the notebook, to avoid unnecessary dependencies.
If your analysis requires a new library that isn't yet part of the shared `pudl-dev` environment, add it to the `devtools/environment.yml` file so that others will get it when they update their environment.

## Related Resources:
Lots of these guidelines are taken directly from Emily Riederer's post: [RMarkdown Driven Development](https://emilyriederer.netlify.app/post/rmarkdown-driven-development/).
For more in depth explanation of the motivations behind this layout, do go check it out!

# Import Libraries
* Because it's very likely that you will be editing the PUDL Python packages or your own local module under development while working in the notebook, use the %autoreload magic with autoreload level 2 to ensure that any changes you've made in those files are always reflected in the code that's running in the notebook.
* Put all import statements at the top of the notebook, so everyone can see what its dependencies are up front, and so that if an import is going to fail, it fails early, before the rest of the notebook is run.
* Try to avoid importing individual functions and classes from deep within packages, as it may not be clear to other users where those elements came from, later in the notebook, and also to minimize the chance of namespace collisions.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Standard libraries
import logging
import os
import pathlib
import sys

# 3rd party libraries
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
import pandas as pd
import seaborn as sns
import sqlalchemy as sa

# Local libraries
import pudl

# Configure Display Parameters

In [3]:
sns.set()
%matplotlib inline
mpl.rcParams['figure.figsize'] = (10,4)
mpl.rcParams['figure.dpi'] = 150
pd.options.display.max_columns = 100
pd.options.display.max_rows = 100

# Use Python Logging facilities
* Using a logger from the beginning will make the transition into the PUDL package easier.
* Creating a logging handler here will also allow you to see the logging output coming from PUDL and other underlying packages.

In [4]:
# logger=logging.getLogger()
# logger.setLevel(logging.INFO)
# handler = logging.StreamHandler(stream=sys.stdout)
# formatter = logging.Formatter('%(message)s')
# handler.setFormatter(formatter)
# logger.handlers = [handler]

# Define Functions
In many cases, the eventual product of a notebook analysis is going to be the creation of new, reusable functions that are integrated into the underlying PUDL code. You should begin the process of accumulating and organizing those functions as soon as you notice repeated patterns in your code.
* Functions should be used to encapsulate any potentially reusable code.
* Functions should be defined immediately after the imports, to avoid accidental dependence on zombie variables that are defined further down in the code
* While they may evolve over time, having brief docstrings explaining what they do will help others understand them.
* If there's a particular type of plot or visualization that is made repeatedly in the notebook, it's good to parameterize and functionalize it here too, so that as you refine the presentation of the data and results, those improvements can be made in a single place, and shown uniformly throughout the notebook.
* As these functions mature and become more general purpose tools, you will probably want to start migrating them into their own local module, under a `src` directory in the same directory as the notebook. You will want to import this module 

## Dummy EIA 861 ETL

In [5]:
def test_etl_eia(eia_inputs, pudl_settings):
    """
    This is a dummy function that runs the first part of the EIA ETL
    process -- everything up until the entity harvesting begins. For
    use in this notebook only.

    """
    eia860_tables = eia_inputs["eia860_tables"]
    eia860_years = eia_inputs["eia860_years"]
    eia861_tables = eia_inputs["eia861_tables"]
    eia861_years = eia_inputs["eia861_years"]
    eia923_tables = eia_inputs["eia923_tables"]
    eia923_years = eia_inputs["eia923_years"]

    # generate CSVs for the static EIA tables, return the list of tables
    #static_tables = _load_static_tables_eia(datapkg_dir)
    
    # For new Datastore args
    pudl_in = pathlib.Path(pudl_settings['pudl_in'])
    ds = pudl.workspace.datastore.Datastore(pudl_in, sandbox=True)
    
    # Extract EIA forms 923, 860
    eia860_raw_dfs = pudl.extract.eia860.Extractor(ds).extract(eia860_years)
    eia861_raw_dfs = pudl.extract.eia861.Extractor(ds).extract(eia861_years)
    eia923_raw_dfs = pudl.extract.eia923.Extractor(ds).extract(eia923_years)

    # Transform EIA forms 860, 861, 923
    eia860_transformed_dfs = pudl.transform.eia860.transform(eia860_raw_dfs, eia860_tables=eia860_tables)
    eia861_transformed_dfs = pudl.transform.eia861.transform(eia861_raw_dfs, eia861_tables=eia861_tables)
    eia923_transformed_dfs = pudl.transform.eia923.transform(eia923_raw_dfs, eia923_tables=eia923_tables)

    # create an eia transformed dfs dictionary
    eia_transformed_dfs = eia860_transformed_dfs.copy()
    eia_transformed_dfs.update(eia861_transformed_dfs.copy())
    eia_transformed_dfs.update(eia923_transformed_dfs.copy())

    # convert types..
    eia_transformed_dfs = pudl.helpers.convert_dfs_dict_dtypes(eia_transformed_dfs, 'eia')

    return eia860_raw_dfs, eia861_raw_dfs, eia923_raw_dfs, eia_transformed_dfs

# Define Notebook Parameters
If there are overarching parameters which determine the nature of the analysis -- which US states to look at, which utilities are of interest, a particular start and end date -- state those clearly at the beginning of the analysis, so that they can be referred to by the rest of the notebook and easily changed if need be.
* If the analysis depends on local (non-PUDL managed) datasets, define the paths to those data here.
* If there are external URLs or other resource locations that will be accessed, define those here as well.
* This is also where you should create your `pudl_settings` dictionary and connections to your local PUDL databases

In [6]:
EIA861_YEARS = list(range(2001, 2019))
pudl_settings = pudl.workspace.setup.get_defaults()
display(pudl_settings)

ferc1_engine = sa.create_engine(pudl_settings['ferc1_db'])
display(ferc1_engine)

pudl_engine = sa.create_engine(pudl_settings['pudl_db'])
display(pudl_engine)

pudl_out = pudl.output.pudltabl.PudlTabl(pudl_engine)


# Is there other external data you need to pull in?
# If so, put it in a (relatively) standard place on the filesystem.
my_new_data_url = "https://mynewdata.website.gov/path/to/new/data.csv"
my_new_datadir = pathlib.Path(pudl_settings["data_dir"]) / "local/new_data_source"

# Store API keys and other secrets in environment variables
# and read them in here, if needed:
# API_KEY_EIA = os.environ["API_KEY_EIA "]
# API_KEY_FRED = os.environ["API_KEY_FRED "]

{'pudl_in': '/Users/aesharpe/Desktop/Work/Catalyst_Coop/PUDL_DIR',
 'data_dir': '/Users/aesharpe/Desktop/Work/Catalyst_Coop/PUDL_DIR/data',
 'settings_dir': '/Users/aesharpe/Desktop/Work/Catalyst_Coop/PUDL_DIR/settings',
 'pudl_out': '/Users/aesharpe/Desktop/Work/Catalyst_Coop/PUDL_DIR',
 'sqlite_dir': '/Users/aesharpe/Desktop/Work/Catalyst_Coop/PUDL_DIR/sqlite',
 'parquet_dir': '/Users/aesharpe/Desktop/Work/Catalyst_Coop/PUDL_DIR/parquet',
 'datapkg_dir': '/Users/aesharpe/Desktop/Work/Catalyst_Coop/PUDL_DIR/datapkg',
 'notebook_dir': '/Users/aesharpe/Desktop/Work/Catalyst_Coop/PUDL_DIR/notebook',
 'ferc1_db': 'sqlite:////Users/aesharpe/Desktop/Work/Catalyst_Coop/PUDL_DIR/sqlite/ferc1.sqlite',
 'pudl_db': 'sqlite:////Users/aesharpe/Desktop/Work/Catalyst_Coop/PUDL_DIR/sqlite/pudl.sqlite'}

Engine(sqlite:////Users/aesharpe/Desktop/Work/Catalyst_Coop/PUDL_DIR/sqlite/ferc1.sqlite)

Engine(sqlite:////Users/aesharpe/Desktop/Work/Catalyst_Coop/PUDL_DIR/sqlite/pudl.sqlite)

# Load Data
* Given the above parameters and functions, it should now be possible to load the required data into local variables for further wrangling, analysis, and visualization.
* If the data is not yet present on the machine of the person running the notebook, this step should also acquire the data from its original source, and place it in the appropriate location under `<PUDL_IN>/data/local/`.
* If there are steps which have to be done manually here, put them first so that they fail first if the user hasn't read the instructions, and they can fix the situation before a bunch of other work gets done. Try to minimize the amount of work in the filesystem that has to be done manually though.
* If this process takes a little while, don't be shy about producing `logging` output.
* Using the `%%time` cell magic can also help users understand which pieces of work / data acquisition are hard:

## EIA 861 (2010-2018)
* Not yet fully integrated into PUDL
* Post-transform harvesting process isn't compatible w/ EIA 861 structure
* Only getting the `sales_eia861`, `balancing_authority_eia861`, and `service_territory_eia861` tables

### Already Transformed EIA 861 DataFrames

In [7]:
%%time
eia_inputs = {
    "eia860_years": [],
    "eia860_tables": pudl.constants.pudl_tables["eia860"],
    "eia861_years": EIA861_YEARS,
    "eia861_tables": pudl.constants.pudl_tables["eia861"],
    "eia923_years": [],
    "eia923_tables": pudl.constants.pudl_tables["eia923"],
}
eia860_raw_dfs, eia861_raw_dfs, eia923_raw_dfs, eia_transformed_dfs = test_etl_eia(eia_inputs=eia_inputs, pudl_settings=pudl_settings)

/Users/aesharpe/Desktop/Work/Catalyst_Coop/pudl/src/pudl/extract/eia861.py:39: UserWarning: Integration of EIA 861 into PUDL is still experimental and incomplete.
The data has not yet been validated, and the structure may change.
  warnings.warn(


WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
The following reported NERC regions are not currently recognized and become UNK values: ['CLECO']
The following reported NER

# Sanity Check Data
If there's any validation that can be done on the data which you've loaded to flag if/when it is inappropriate for the analysis that follows, do it here. If you find the data is unusable, use `assert` statements or `raise` Exceptions to stop the notebook from proceeding, and indicate what the problem is.

In [19]:
df = eia861_raw_dfs['reliability_eia861'].copy()
df2 = eia_transformed_dfs['reliability_eia861'].copy()
# other_standard_momentary_interruption_definition

In [20]:
df2

,utility_id_eia,state,report_date,entity_type,utility_name_eia,reliability_standard,caidi_w_major_event_days_minus_loss_of_service_minutes,caidi_w_major_event_days_minutes,caidi_wo_major_event_days_minutes,customers,highest_distribution_voltage_kv,inactive_accounts_included,momentary_interruption_definition,outages_recorded_automatically,saidi_w_major_event_days_minus_loss_of_service_minutes,saidi_w_major_event_days_minutes,saidi_wo_major_event_days_minutes,saifi_w_major_event_days_customers,saifi_w_major_event_days_minus_loss_of_service_customers,saifi_wo_major_event_days_customers
0,97,IL,2013-01-01,NaN,Adams Electric Coop,ieee_standard,NaN,NaN,NaN,8715.0,34.5,<NA>,NaN,False,191.0,223.00,158.00,1.810,1.390,1.350
1,97,IL,2013-01-01,NaN,Adams Electric Coop,other_standard,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN
2,108,WI,2013-01-01,NaN,Adams-Columbia Electric Coop,ieee_standard,NaN,NaN,NaN,36014.0,12.0,<NA>,NaN,True,139.0,143.00,58.00,1.093,1.028,0.691
3,108,WI,2013-01-01,NaN,Adams-Columbia Electric Coop,other_standard,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN
4,155,MN,2013-01-01,NaN,Agralite Electric Coop,ieee_standard,NaN,NaN,NaN,5122.0,12.0,<NA>,NaN,False,1134.0,1142.00,141.00,3.000,2.000,1.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13085,60482,OK,2018-01-01,Cooperative,CKenergy Electric Cooperative,other_standard,NaN,108.9491,92.304321,24757.0,24.9,False,Less than 1 minute,True,NaN,363.89,294.82,3.340,NaN,3.194
13086,60631,MI,2018-01-01,Investor Owned,Upper Michigan Energy Resources Corp.,ieee_standard,211.2069,211.2069,160.439560,37673.0,25.0,<NA>,NaN,False,245.0,245.00,146.00,1.160,1.160,0.910
13087,60631,MI,2018-01-01,Investor Owned,Upper Michigan Energy Resources Corp.,other_standard,NaN,NaN,NaN,NaN,NaN,<NA>,NaN,<NA>,NaN,NaN,NaN,NaN,NaN,NaN
13088,60839,KS,2018-01-01,Cooperative,Southern Pioneer Electric Company,ieee_standard,NaN,NaN,NaN,16870.0,13.8,<NA>,NaN,True,132.0,132.00,132.00,NaN,NaN,NaN


In [18]:
test = pudl.transform.eia861._compare_nerc_physical_w_nerc_operational(df)
#test[(test['nerc_region']!='UNK') & (test['nerc_regions_of_operation']!= 'UNK')]


KeyError: 'nerc_region'

In [11]:
df.columns.tolist()

['utility_id_eia',
 'state',
 'report_date',
 'entity_type',
 'utility_name_eia',
 'reliability_standard',
 'caidi_w_major_event_days_minus_loss_of_service_minutes',
 'caidi_w_major_event_days_minutes',
 'caidi_wo_major_event_days_minutes',
 'customers',
 'highest_distribution_voltage_kv',
 'inactive_accounts_included',
 'momentary_interruption_definition',
 'outages_recorded_automatically',
 'saidi_w_major_event_days_minus_loss_of_service_minutes',
 'saidi_w_major_event_days_minutes',
 'saidi_wo_major_event_days_minutes',
 'saifi_w_major_event_days_customers',
 'saifi_w_major_event_days_minus_loss_of_service_customers',
 'saifi_wo_major_event_days_customers']

In [19]:
df

,utility_id_eia,state,report_date,nerc_region,entity_type,utility_name_eia,alternative_fuel_vehicle_2_activity,alternative_fuel_vehicle_activity,bundled_activity,buy_distribution_activity,buying_transmission_activity,distribution_activity,generation_activity,operates_generating_plant,retail_marketing_activity,transmission_activity,wholesale_marketing_activity
0,34,SC,2007-01-01,SERC,NaN,Abbeville City of,False,False,False,False,False,True,True,True,False,False,False
1,55,MS,2007-01-01,SERC,NaN,Aberdeen City of,False,False,False,False,False,True,False,False,False,False,False
2,59,LA,2007-01-01,SPP,NaN,City of Abbeville,False,False,False,False,False,True,False,False,False,False,False
3,84,VA,2007-01-01,RFC,NaN,A & N Electric Coop,False,False,False,False,False,True,True,True,False,True,False
4,87,MN,2007-01-01,MRO,NaN,City of Ada,False,False,False,False,False,True,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32098,61769,TX,2018-01-01,UNK,Retail Power Marketer,GridPlus Texas Inc,False,False,False,False,False,False,False,False,True,False,False
32099,61823,CT,2018-01-01,UNK,Behind the Meter,"Greenskies Renewable Energy, LLC",False,False,False,False,False,False,False,False,True,False,False
32100,61858,CA,2018-01-01,WECC,Community Choice Aggregator,San Jose Clean Energy,False,False,False,False,False,False,False,False,True,False,True
32101,62107,CA,2018-01-01,UNK,Municipal,City of Industry,False,False,False,False,False,True,False,False,False,False,False


In [138]:
# Set NA states to UNK
df['state'] = df['state'].fillna('UNK')

# Create column indicating whether the nerc region matches the nerc region of operation TRUE
df['nerc_match'] = df['nerc_region'] == df['nerc_regions_of_operation']

# Group by utility, state, and report date to see which groups have at least one match 
grouped_nerc_match_bools = (
    df.groupby(['utility_id_eia', 'state', 'report_date'])
    [['nerc_match']].any()
    .reset_index()
    .rename(columns={'nerc_match': 'nerc_group_match'})
)
expanded_nerc_match_bools = (
    pd.merge(df, 
             grouped_nerc_match_bools, 
             on=['utility_id_eia', 'state', 'report_date'], 
             how='left')
)

expanded_nerc_match_bools_false = expanded_nerc_match_bools[~expanded_nerc_match_bools['nerc_group_match']]

In [139]:
expanded_nerc_match_bools_false

,utility_id_eia,state,report_date,nerc_region,nerc_regions_of_operation,nerc_match,nerc_group_match
9,149,IA,2007-01-01,MRO,RFC,False,False
25,261,IN,2007-01-01,ECAR,RFC,False,False
43,406,IL,2007-01-01,RFC,SERC,False,False
80,856,TX,2007-01-01,UNK,TRE,False,False
87,925,AR,2007-01-01,SERC,SPP,False,False
...,...,...,...,...,...,...,...
28145,61475,CA,2018-01-01,UNK,WECC,False,False
28146,61503,CA,2018-01-01,UNK,WECC,False,False
28147,61570,CA,2018-01-01,UNK,WECC,False,False
28148,61769,TX,2018-01-01,UNK,TRE,False,False


In [132]:
df[df['utility_id_eia']==56739]

,utility_id_eia,state,report_date,nerc_region,nerc_regions_of_operation,nerc_match
104735,56739,<NA>,2010-01-01,UNK,WECC,False
130823,56739,AK,2011-01-01,UNK,WECC,False


In [72]:
idx_cols = [
    'utility_id_eia',
    'state',
    'report_date',
    'nerc_region'
]

nerc_class = [
    'tre',
    'frcc',
    'mro',
    'npcc',
    'rfc',
    'serc',
    'spp',
    'wecc',
]

rto_class = [
    'caiso',
    'ercot',
    'pjm',
    'nyiso',
    'spp',
    'miso',
    'isone',
    'other'
]

In [34]:
nerc_cols = [col for col in raw_ud if 'nerc_region_operation' in col]
rto_cols = [col for col in raw_ud if 'rto_operation' in col]

raw_ud_nerc = raw_ud[idx_cols + nerc_cols].copy()
raw_ud_rto = raw_ud[idx_cols + rto_cols].copy()
raw_ud_misc = raw_ud.drop(nerc_cols + rto_cols, axis=1).copy()

In [82]:
# Replace all Y/N/Nan values with boolean
transformed_ud_misc_bool = (
    raw_ud_misc
    .drop(['entity_type', 'utility_name_eia'], axis=1)
    .set_index(idx_cols)
    .fillna(False)
    .replace({"N": False, "Y": True})
)
# Merge misc. bool cols back together with misc. non bool cols
transformed_ud_misc = (
    pd.merge(
        raw_ud_misc[idx_cols+['entity_type', 'utility_name_eia']],
        transformed_ud_misc_bool,
        on=idx_cols,
        how='outer'
    )
)

32103 32103


In [35]:
tidy_ud_nerc, nerc_idx_cols = pudl.transform.eia861._tidy_class_dfs(
    df=raw_ud_nerc,
    df_name='Utility Data NERC Regions',
    idx_cols=idx_cols,
    class_list=nerc_class,
    class_type='nerc_region_operational',
)

In [61]:
transformed_ud_nerc = (
    tidy_ud_nerc.assign(
        nerc_region_operation=lambda x: (
            x.nerc_region_operation
            .fillna(False)
            .replace({"N": False, "Y": True})),
        nerc_region_operational=lambda x: (
            x.nerc_region_operational.str.upper()
        )
    )
)


# Only keep true values and drop bool col
transformed_ud_nerc = (
    transformed_ud_nerc[transformed_ud_nerc.nerc_region_operation]
    .drop(['nerc_region_operation'], axis=1)
)

In [62]:
transformed_ud_nerc

,utility_id_eia,state,report_date,nerc_region_physical,nerc_region_operational,nerc_region_operation
4,34,SC,2007-01-01,SERC,SERC,True
21,59,LA,2007-01-01,SPP,SPP,True
27,84,VA,2007-01-01,RFC,RFC,True
33,87,MN,2007-01-01,MRO,MRO,True
44,97,IL,2007-01-01,SERC,SERC,True
...,...,...,...,...,...,...
256759,61503,CA,2018-01-01,UNK,WECC,True
256783,61570,CA,2018-01-01,UNK,WECC,True
256790,61769,TX,2018-01-01,UNK,TRE,True
256807,61858,CA,2018-01-01,WECC,WECC,True


In [25]:
transformed_ud_nerc

,utility_id_eia,state,report_date,nerc_region_physical,nerc_region_operational,nerc_region_operation
0,34,SC,2007-01-01,SERC,frcc,False
1,34,SC,2007-01-01,SERC,mro,False
2,34,SC,2007-01-01,SERC,npcc,False
3,34,SC,2007-01-01,SERC,rfc,False
4,34,SC,2007-01-01,SERC,serc,True
...,...,...,...,...,...,...
256819,62776,CN,2018-01-01,UNK,rfc,False
256820,62776,CN,2018-01-01,UNK,serc,False
256821,62776,CN,2018-01-01,UNK,spp,False
256822,62776,CN,2018-01-01,UNK,tre,False


In [ ]:
denorm_cols = pudl.transform.eia861._filter_non_class_cols(raw_df, class_list).reset_index()

# Merge the index, data, and denormalized columns back together
tidy_df = pd.merge(denorm_cols, data_cols, on=idx_cols)

# Compare reported totals with sum of component columns
if 'total' in class_list:
    _compare_totals(data_cols, idx_cols, class_type, df_name)
if keep_totals is False:
    tidy_df = tidy_df.query(f"{class_type}!='total'")

In [14]:
raw_ud_nerc

,utility_id_eia,state,report_date,nerc_region,operating_in_nerc_mro,operating_in_nerc_npcc,operating_in_nerc_rfc,operating_in_nerc_serc,operating_in_nerc_spp,operating_in_nerc_wecc
0,34,SC,2007-01-01,SERC,NaN,NaN,NaN,Y,NaN,NaN
1,55,MS,2007-01-01,SERC,NaN,NaN,NaN,NaN,NaN,NaN
2,59,LA,2007-01-01,SPP,NaN,NaN,NaN,NaN,Y,NaN
3,84,VA,2007-01-01,RFC,NaN,NaN,Y,NaN,NaN,NaN
4,87,MN,2007-01-01,MRO,Y,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
32128,62107,CA,2018-01-01,NaN,NaN,NaN,NaN,NaN,NaN,Y
32129,62776,CN,2018-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32130,88888,TX,2018-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN
32131,88888,TX,2018-01-01,TRE,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
test_df.columns.tolist()

['alternative_fuel_vehicle_2_activity',
 'alternative_fuel_vehicle_activity',
 'bundled_activity',
 'buy_distribution_activity',
 'buying_transmission_activity',
 'distribution_activity',
 'entity_type',
 'generation_activity',
 'nerc_region',
 'operates_generating_plant',
 'operating_in_caiso',
 'operating_in_ercot',
 'operating_in_frcc',
 'operating_in_isone',
 'operating_in_miso',
 'operating_in_mro',
 'operating_in_npcc',
 'operating_in_nyiso',
 'operating_in_other',
 'operating_in_pjm',
 'operating_in_rfc',
 'operating_in_serc',
 'operating_in_spp_nerc',
 'operating_in_spp_rto',
 'operating_in_tre',
 'operating_in_wecc',
 'retail_marketing_activity',
 'state',
 'transmission_activity',
 'utility_id_eia',
 'utility_name_eia',
 'wholesale_marketing_activity',
 'report_date']

In [71]:
STANDARDS = [
    'ieee',
    'other'
]

idx_cols = [
    'utility_id_eia',
    'state',
    'report_date'
]

df = test_df

In [72]:
#tidy_df, idx_cols = pudl.transform.eia861._tidy_class_dfs(
df=test_df
df_name='Reliability'
idx_cols=idx_cols
class_list=STANDARDS
class_type='standard'
keep_totals=False
#)

In [73]:
if 'balancing_authority_code_eia' in idx_cols:
    df = (
        df.assign(
            balancing_authority_code_eia=lambda x: x.balancing_authority_code_eia.fillna("UNK"))
    )
raw_df = (
    df.dropna(subset=["utility_id_eia"])
    .astype({"utility_id_eia": pd.Int64Dtype()})
    .set_index(idx_cols)
)
# Split the table into index, data, and "denormalized" columns for processing:
# Separate customer classes and reported data into a hierarchical index
#logger.debug(f"Stacking EIA861 {df_name} data columns by {class_type}.")
data_cols = pudl.transform.eia861._filter_class_cols(raw_df, class_list)
data_cols.columns = (
    data_cols.columns.str.split("_", n=1, expand=True)
    .set_names([class_type, None])
)
# Now stack the customer classes into their own categorical column,
data_cols = (
    data_cols.stack(level=0, dropna=False)
    .reset_index()
)
denorm_cols = pudl.transform.eia861._filter_non_class_cols(raw_df, class_list).reset_index()

# Merge the index, data, and denormalized columns back together
tidy_df = pd.merge(denorm_cols, data_cols, on=idx_cols)

# Compare reported totals with sum of component columns
if 'total' in class_list:
    pudl.transform.eia861._compare_totals(data_cols, idx_cols, class_type, df_name)

# Remove the now redundant "Total" records -- they can be reconstructed
# from the other customer classes.
if not keep_totals:
    tidy_df = tidy_df.query(f"{class_type}!='total'")

In [74]:
tidy_df

,utility_id_eia,state,report_date,entity_type,utility_name_eia,standard,caidi_with_med,caidi_with_med_minus_los,caidi_without_med,customers,highest_distribution_voltage,inactive_accounts_included,momentary_interruptions,outages_recorded_automatically,saidi_with_med,saidi_with_med_minus_los,saidi_without_med,saifi_with_med,saifi_with_med_minus_los,saifi_without_med
0,97,IL,2013-01-01,NaN,Adams Electric Coop,ieee,NaN,NaN,NaN,8715.0,34.5,NaN,NaN,N,223.00,191.0,158.00,1.810,1.390,1.350
1,97,IL,2013-01-01,NaN,Adams Electric Coop,other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,108,WI,2013-01-01,NaN,Adams-Columbia Electric Coop,ieee,NaN,NaN,NaN,36014.0,12.0,NaN,NaN,Y,143.00,139.0,58.00,1.093,1.028,0.691
3,108,WI,2013-01-01,NaN,Adams-Columbia Electric Coop,other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,155,MN,2013-01-01,NaN,Agralite Electric Coop,ieee,NaN,NaN,NaN,5122.0,12.0,NaN,NaN,N,1142.00,1134.0,141.00,3.000,2.000,1.000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13085,60482,OK,2018-01-01,Cooperative,CKenergy Electric Cooperative,other,108.9491,NaN,92.304321,24757.0,24.9,N,L,Y,363.89,NaN,294.82,3.340,NaN,3.194
13086,60631,MI,2018-01-01,Investor Owned,Upper Michigan Energy Resources Corp.,ieee,211.2069,211.2069,160.439560,37673.0,25.0,NaN,NaN,N,245.00,245.0,146.00,1.160,1.160,0.910
13087,60631,MI,2018-01-01,Investor Owned,Upper Michigan Energy Resources Corp.,other,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13088,60839,KS,2018-01-01,Cooperative,Southern Pioneer Electric Company,ieee,NaN,NaN,NaN,16870.0,13.8,NaN,NaN,Y,132.00,132.0,132.00,NaN,NaN,NaN


# Preliminary Data Wrangling
Once all of the data is loaded and looks like it's in good shape, do any initial wrangling that's specific to this particular analysis. This should mostly make use of the higher level functions which were defined above. If this step takes a while, don't be shy about producing `logging` outputs.

In [8]:
idx_cols = [
    'utility_id_eia',
    'state',
    'balancing_authority_code_eia',
    'report_date',
]

CUSTOMER_CLASSES = [
    "commercial",
    "dircnct",  # previously direct_connection
    "industrial",
    "other",
    "residential",
    "total",
    "transportation"
]

# 1495 w/0 ba
# 1489 w/ ba

In [10]:
df = eia_transformed_dfs['non_net_metering_eia861'].copy()
df2 = eia861_raw_dfs['non_net_metering_eia861'].copy()
df = pudl.helpers.organize_cols(df, idx_cols+['utility_name_eia','customer_class', 'tech_class'])
#df[df['7'].notnull()]
df

,utility_id_eia,state,balancing_authority_code_eia,report_date,utility_name_eia,customer_class,tech_class,backup_capacity_mw,capacity_mw,current_flow_type,generators_number,utility_owned_capacity_mw
0,55,MS,TVA,2016-01-01,City of Aberdeen - (MS),commercial,combturb,NaN,NaN,NaN,1.0,NaN
1,55,MS,TVA,2016-01-01,City of Aberdeen - (MS),commercial,fcell,NaN,NaN,NaN,1.0,NaN
2,55,MS,TVA,2016-01-01,City of Aberdeen - (MS),commercial,hydro,NaN,NaN,NaN,1.0,NaN
3,55,MS,TVA,2016-01-01,City of Aberdeen - (MS),commercial,ice,NaN,NaN,NaN,1.0,NaN
4,55,MS,TVA,2016-01-01,City of Aberdeen - (MS),commercial,other,NaN,NaN,NaN,1.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
90775,99999,WY,PACE,2018-01-01,Adjustment 2018,transportation,pv,NaN,NaN,NaN,NaN,NaN
90776,99999,WY,PACE,2018-01-01,Adjustment 2018,transportation,steam,NaN,NaN,NaN,NaN,NaN
90777,99999,WY,PACE,2018-01-01,Adjustment 2018,transportation,storage,NaN,NaN,NaN,NaN,NaN
90778,99999,WY,PACE,2018-01-01,Adjustment 2018,transportation,total,NaN,NaN,NaN,NaN,NaN


In [14]:
df2.columns.tolist()

['backup_capacity_mw',
 'balancing_authority_code_eia',
 'commercial_combturb_capacity_mw',
 'commercial_fcell_capacity_mw',
 'commercial_hydro_capacity_mw',
 'commercial_ice_capacity_mw',
 'commercial_other_capacity_mw',
 'commercial_pv_capacity_mw',
 'commercial_storage_capacity_mw',
 'commercial_total_capacity_mw',
 'commercial_wind_capacity_mw',
 'dircnct_combturb_capacity_mw',
 'dircnct_fcell_capacity_mw',
 'dircnct_hydro_capacity_mw',
 'dircnct_ice_capacity_mw',
 'dircnct_other_capacity_mw',
 'dircnct_pv_capacity_mw',
 'dircnct_steam_capacity_mw',
 'dircnct_storage_capacity_mw',
 'dircnct_total_capacity_mw',
 'dircnct_wind_capacity_mw',
 'generators_number',
 'industrial_combturb_capacity_mw',
 'industrial_fcell_capacity_mw',
 'industrial_hydro_capacity_mw',
 'industrial_ice_capacity_mw',
 'industrial_other_capacity_mw',
 'industrial_pv_capacity_mw',
 'industrial_steam_capacity_mw',
 'industrial_storage_capacity_mw',
 'industrial_total_capacity_mw',
 'industrial_wind_capacity_mw'

In [17]:
# # Define base comparison df 
# eia_base_df = (
#     pudl_out.utils_eia860()
#     #[['report_date', 'utility_id_eia', 'utility_id_pudl', 'utility_name_eia', 'state']]
#     .assign(utility_id_eia=lambda x: x.utility_id_eia.astype('Int64'))
# )
# # Shorten df
# #df = df[['utility_id_eia', 'state', 'utility_name_eia', 'report_date']]

# # Merge dfs together 
# df_merge = (
#     pd.merge(eia_base_df, df, on=['utility_id_eia', 'state', 'report_date'], how='outer', suffixes=('_base', '_861'))
#     .set_index(['utility_id_eia', 'report_date'])
#     .assign(utility_name_eia_base=lambda x: x.utility_name_eia_base.astype('string'),
#             utility_name_eia_861=lambda x: x.utility_name_eia_861.astype('string'))
# )
# df_merge

## Fuzzy Match Columns

In [85]:
from fuzzywuzzy import fuzz

def fuzzy_match(df, col1, col2):
    # Define base comparison df 
    eia_base_df = (
        pudl_out.utils_eia860()
        #[['report_date', 'utility_id_eia', 'utility_id_pudl', 'utility_name_eia', 'state']]
        .assign(utility_id_eia=lambda x: x.utility_id_eia.astype('Int64'))
    )
    # Shorten df
    #df = df[['utility_id_eia', 'state', 'utility_name_eia', 'report_date']]
    
    # Merge dfs together 
    df_merge = (
        pd.merge(eia_base_df, df, on=['utility_id_eia', 'state'], how='outer', suffixes=('_base', '_861'))
        .set_index(['utility_id_eia', 'state'])
        .assign(utility_name_eia_base=lambda x: x.utility_name_eia_base.astype('string'),
                utility_name_eia_861=lambda x: x.utility_name_eia_861.astype('string'))
    )
    
    # Only run where both columns for utility name have values
    df_merge_no_null = (
        df_merge.loc[
            (df_merge[col1].notnull()) 
            & (df_merge[col2].notnull())].copy()
    )
    
    # Fuzzy match
    df_merge_no_null['fuzzy'] = (
        df_merge_no_null.apply(lambda x: fuzz.ratio(x.utility_name_eia_base, x.new_parent),axis=1)
    )
#     df_merge_no_null = (
#         df_merge_no_null.assign(fuzzy=lambda x: fuzz.ratio(x[col1], x[col2])))
    
    #df_merge_no_null.loc[:,'fuzzy'] = fuzzy_series
    df_merge_no_null = df_merge_no_null.sort_values('fuzzy')
    
    return df_merge_no_null

# DSM Stuff

In [37]:
dsm = eia861_raw_dfs['demand_side_management_eia861'].copy()
#dsm.columns.tolist()

In [50]:
idx_cols = ['utility_id_eia', 'state', 'ba_code', 'report_year']

dsm_test = dsm[['utility_id_eia', 'state', 'ba_code', 'report_year', 
                'residential_annual_load_management_energy_effects ', 
                'commercial_annual_load_management_energy_effects ', 
                'industrial_annual_load_management_energy_effects ',
                'transportation_annual_load_management_energy_effects ',
                'total_annual_load_management_energy_effects ',
]]

dr_test = raw_dr_eia861[['utility_id_eia', 'state', 'balancing_authority_code_eia', 'report_year',
                         'residential_energy_savings_mwh',
                         'commercial_energy_savings_mwh',
                         'industrial_energy_savings_mwh',
                         'transportation_energy_savings_mwh',
                         'total_energy_savings_mwh'
]].rename(columns={'balancing_authority_code_eia': 'ba_code'})

In [84]:
merge = pd.merge(dsm_test, dr_test, on=idx_cols, how='outer')
merge = pudl.helpers.oob_to_nan(merge, 
                       ['residential_annual_load_management_energy_effects ', 
                        'commercial_annual_load_management_energy_effects ', 
                        'industrial_annual_load_management_energy_effects ',
                        'transportation_annual_load_management_energy_effects ',
                        'residential_energy_savings_mwh',
                        'commercial_energy_savings_mwh',
                        'industrial_energy_savings_mwh',
                        'transportation_energy_savings_mwh',
                        'total_energy_savings_mwh',
                        'total_annual_load_management_energy_effects '])

In [71]:
dr_col = 'residential_energy_savings_mwh'
dsm_col = 'residential_annual_load_management_energy_effects '

In [36]:
merge = merge.loc[merge['report_year'].isin(range(2012,2014))]
merge = merge.loc[merge[dr_col].notna()]
#merge = merge.loc[merge[dsm_col].notna()]


NameError: name 'merge' is not defined

In [34]:
#test.plot.scatter('residential_annual_load_management_energy_effects ', 'residential_energy_savings_mwh')

In [33]:
#raw_dr_eia861.columns.tolist()

# Data Analysis and Visualization
* Now that you've got the required data in a usable form, you can tell the story of your analysis through a mix of visualizations, and further data wrangling steps.
* This narrative should be readable, with figures that have titles, legends, and labeled axes as appropriate so others can understand what you're showing them.
* The code should be concise and make use of the parameters and functions which you've defined above when possible. Functions should contain comprehensible chunks of work that make sense as one step in the story of the analysis.